In [1]:
using Revise

In [2]:
using MacroTools

In [3]:
using Boolean

Precompiling Boolean
  ✓ Boolean
  1 dependency successfully precompiled in 1 seconds. 30 already precompiled.
[ Info: Precompiling Boolean [top-level]


In [4]:
init_logic(3)
println(simplifyLogic(Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)")))

x2 * (x3 * (x1 + x2))


In [5]:
create_bool_rep("z1 + z2")

Formula    = z1 + z2
Variable   = z
Size       = 3
Bit vector = Bool[0, 1, 1, 1, 0, 1, 1, 1]


In [6]:
z = simplifyLogic(Meta.parse("x1 * (1 ⊕ x1)"))

:(x1 * ~x1)

In [7]:
z.args[3].args

2-element Vector{Any}:
 :~
 :x1

In [8]:
BitArray([((i-1) >> (j-1)) & 1  for i in 1:2^3, j in 1:3])

8×3 BitMatrix:
 0  0  0
 1  0  0
 0  1  0
 1  1  0
 0  0  1
 1  0  1
 0  1  1
 1  1  1

In [9]:
simplifyLogic(Meta.parse("(z1 ⊕ z3) ⊕ (0 ⊕ z3)"))

:(z3 ⊕ (z1 ⊕ z3))

In [10]:
simplifyLogic(Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)"))

:(x2 * (x3 * (x1 + x2)))

In [11]:
simplifyLogic(Meta.parse("0 + x1"))

:x1

In [12]:
simplifyLogic(Meta.parse("x1 * (x1 + 0)")) 

:x1

In [13]:
init_logic(3)

f1 = "x1 + x2 * x3"
f2 = "(z1 * z3) ⊕ (z2 * z3)"
zero = "((x1 + x2) * x3) ⊕ (x1 * x3 + x2 * x3)"

println("\nExamining the 'f2' formula:")
f2e = create_bool_rep(f2)
println(f2e)
print("Number of non-zero bits: ")
println(logicCount(f2e))
println("First 5 (at most 5) variable values that lead to a 'true' value are:")
println("$(f2e.var)1, $(f2e.var)2, $(f2e.var)3):")
res = nonZero(f2e, head=5)
print(res)

println("\nExamining the 'zero' formula:")
zeroe = create_bool_rep(zero)
println(zeroe)
print("Number of non-zero bits: ")
println(logicCount(zeroe))
println("First 5 (at most 5) variable values that lead to a 'true' value are:")
res = nonZero(zeroe, head=5)
typeof(res)
res
logicCount(zeroe)


Examining the 'f2' formula:
Formula    = (z1 * z3) ⊕ (z2 * z3)
Variable   = z
Size       = 3
Bit vector = Bool[0, 0, 0, 0, 0, 1, 1, 0]

Number of non-zero bits: 2
First 5 (at most 5) variable values that lead to a 'true' value are:
z1, z2, z3):
(1, 0, 1)
(0, 1, 1)

Examining the 'zero' formula:
Formula    = ((x1 + x2) * x3) ⊕ (x1 * x3 + x2 * x3)
Variable   = x
Size       = 3
Bit vector = Bool[0, 0, 0, 0, 0, 0, 0, 0]

Number of non-zero bits: 0
First 5 (at most 5) variable values that lead to a 'true' value are:


0

In [14]:
create_bool_rep("(z1 + z2) * z3")

Formula    = (z1 + z2) * z3
Variable   = z
Size       = 3
Bit vector = Bool[0, 0, 0, 0, 0, 1, 1, 1]


In [15]:
isEquiv(f1, f1)

true

In [16]:
Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)")

:(((x1 + x2) * x3) * (x1 * 0 + x2 * 1))

In [17]:
simplifyLogic(Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)"))

:(x2 * (x3 * (x1 + x2)))

In [18]:
frm="((1 + x2) * x3) * x2"
ee = simplifyLogic(Meta.parse(frm))
string(ee)

"x2 * x3"

In [19]:
    x = reverse(bitstring(UInt8(11)))

"11010000"

In [20]:
simplifyLogic(Meta.parse("x1 ⊕ 0 ⊕ x1 ⊕ 1 ⊕ x1"))

:(~x1)

### Operator Precedence: XOR lower than "*" (Both left associative)

In [21]:
simplifyLogic(Meta.parse("1 ⊕ 1 * 0"))

1

In [22]:
simplifyLogic(Meta.parse("(1 ⊕ 1) * 0"))

0

### Operator Precedence: XOR same as "+" (Both left associative)

#### Is "+" higher than XOR: NO

In [23]:
simplifyLogic(Meta.parse("1 ⊕ 1 + 1"))

1

In [24]:
simplifyLogic(Meta.parse("1 ⊕ (1 + 1)"))

0

#### Is XOR higher than "+": NO

In [25]:
simplifyLogic(Meta.parse("1 + 1 ⊕ 1"))

0

In [26]:
simplifyLogic(Meta.parse("1 + (1 ⊕ 1)"))

1

### Extending the Function nonZero
For future improvement of the package. 

In [27]:
# Intension to take a formula and find inputs that return 1.
form="(x1 + (x2 * ~x5)) * (x3 + x4) +  ~x2"

# Total number of variables.
N=5
NV=3

# Init the logic system for (N-NV) variables.
init_logic(N-NV)

x = Vector{Int64}(undef, N)

# For each of the first `NV` variables
# loop through all values of `0` and `1`s.
# For each set of NV values, substitute these into `form`
# and then simply the formula.
# Remap the variables that remain to 1 to (N-NV).

d = Dict{String,String}()
for i in (NV+1):N
    d["x$i"] = "x$(i-NV)"
end

println(d)

for i in 0:(2^NV-1)
    x = reverse(bitstring(UInt8(i)))
    println("\tx = $(x[1:N])")
    frm = form
    firstBits = x[1:NV]
    for j in 1:NV
        frm = replace(frm, "x$j" => x[j])
    end
    
    eqn = simplifyLogic(Meta.parse(frm))
    if eqn == 0 || eqn == 1
        if eqn == 1
            println(x)
        end
    else
        eqn = string(eqn)
        # We need to convert the vars over to 1:(N-NV). FIXME
        for k in keys(d)
            eqn = replace(eqn, k => d[k])
        end
        println("eqn = $eqn")
        eqnb = create_bool_rep(string(eqn))
        if logicCount(eqnb) != 0
            res = nonZero(eqnb, head=5)
            println(typeof(res))
            print(string(res))
        end
    end
end

Dict("x5" => "x2", "x4" => "x1")
	x = 00000
00000000
	x = 10000
10000000
	x = 01000
eqn = x1 * ~x2
BitMatrix
(1, 0)
	x = 11000
eqn = x1
BitMatrix
(1, 0)
(1, 1)
	x = 00100
00100000
	x = 10100
10100000
	x = 01100
eqn = ~x2
BitMatrix
(0, 0)
(1, 0)
	x = 11100
11100000


### Work with bfunc Macro

In [28]:
@bfunc x1 + x2 * x3

Formula    = x1  +  x2  *  x3
Variable   = x
Size       = 3
Bit vector = Bool[0, 1, 0, 1, 0, 1, 1, 1]


In [30]:
@bfunc begin
    (z1 + z2) * z3
    ⟹ 
    z4 + z5
end

Formula    = (z1  +  z2)  *  z3  ⟹  z4  +  z5
Variable   = z
Size       = 5
Bit vector = Bool[1, 1, 1, 1, 1, 0, 0, 0, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [32]:
ex = Meta.parse("(z1 + x2) * z3 ⟹ z4 + z5")

:((z1 + x2) * z3 ⟹ z4 + z5)

In [33]:
ex.head

:call

In [34]:
ex.args

3-element Vector{Any}:
 :⟹
 :((z1 + x2) * z3)
 :(z4 + z5)

In [36]:
ex.args[2].args

3-element Vector{Any}:
 :*
 :(z1 + x2)
 :z3

In [38]:
f = @bfunc x1 + x2 ⟹ x3

Formula    = x1  +  x2  ⟹  x3
Variable   = x
Size       = 3
Bit vector = Bool[1, 0, 0, 0, 1, 1, 1, 1]


In [39]:
f.val

8-element BitVector:
 1
 0
 0
 0
 1
 1
 1
 1

In [40]:
f = @bfunc begin
    x1 + x2
    ⟹
    x2 * x3
end

Formula    = x1  +  x2  ⟹  x2  *  x3
Variable   = x
Size       = 3
Bit vector = Bool[1, 0, 0, 0, 1, 0, 1, 1]


In [41]:
@bfunc (x1 + x2) * x3

Formula    = (x1  +  x2)  *  x3
Variable   = x
Size       = 3
Bit vector = Bool[0, 0, 0, 0, 0, 1, 1, 1]


In [42]:
@bfunc x1 + x2 * x2 + x3

Formula    = x1  +  x2  *  x2  +  x3
Variable   = x
Size       = 3
Bit vector = Bool[0, 1, 1, 1, 1, 1, 1, 1]
